In [1]:
''' Simulation methods for N(t), Q(t) and \lambda(t) '''


def g_exp(i,u,alpha):
    ''' Exponential decay function '''
    return np.exp(-alpha[i]*u)

def simulate_NL(t, fixed_params, model_params):
    ''' Simulation of the d-dimensional Hawkes process: the counting process and the intensity process.
        Returns the d-dimensional vectors of event arrival times N(t) = (N_1(t),...,N_d(t)) and
        the associated sample path of \lambda(t) = (\lambda_1(t),...,\lambda_d(t)). '''
    
    include_lambda = fixed_params[0]
    M_ival = fixed_params[1]
    d = model_params[0]
    l_inf = model_params[1]
    beta = model_params[2]
    alpha = model_params[3]
    mu = model_params[4]
    
    # store arrival times in each dimension and sort time
    immigrant_times = []
    
    for j in range(d):
        immigrants = np.random.poisson(l_inf[j]*t)
        immigrant_times.append(np.sort(np.random.uniform(0,t,immigrants)))
        immigrant_times[j] = immigrant_times[j].tolist()
    
    # store all events (immigrants + accepted offspring) in array
    total_events = immigrant_times
    
    # each generation produces next generation offspring
    next_gen = immigrant_times

    # store all jump sizes of events in array
    total_jump_sizes = [dict() for i in range(d)]
    
    # set counter to loop while there are new generations
    count = 1
    while count > 0: 
        
        # keep track of generated offspring
        accepted_offspring = [[] for k in range(d)]
        
        # loop over all source components
        for j in range(d):
            
            # loop over each event 
            for m in range(len(next_gen[j])):
                
                # initialize B matrix
                B = np.zeros((d,d))
                
                # loop over all target components
                for i in range(d):
                    
                    # start from arrival time parent and generate offspring until t
                    s = next_gen[j][m]
                    
                    # in case of random jump size: draw here (uncomment)
                    if d == 2:                           
                        B[i][j] = np.random.exponential(B_bi[i][j])
                    elif d == 3:
                        B[i][j] = np.random.exponential(B_tri[i][j])
                    
                    # in case of deterministic jump size: draw here (uncomment)
                    #B[i][j] = beta[i][j]
                    
                    # store jumps for constructing \lambda(.) later
                    total_jump_sizes[i][str(s)] = B[i][j]
                    
                    # lambda upper bound from i<-j                   
                    lambda_max = 10**(-5) + B[i][j]*g_exp(i,0,alpha)
                    
                    # loop to generate offspring according to thinning procedure
                    while (s < t):
                        E = np.random.exponential(1/lambda_max)
                        s = s + E
                        if np.random.uniform(0,1)*lambda_max <= B[i][j]*g_exp(i, s-next_gen[j][m], alpha):
                            accepted_offspring[i].append(s)
                    
                    # remove events after t
                    if len(accepted_offspring[i])> 0 and accepted_offspring[i][-1] > t:
                        accepted_offspring[i] = accepted_offspring[i][:-1]                        

        # new generations
        next_gen = accepted_offspring
        
        # add to total number of events
        for j in range(d):
            if len(next_gen[j]) > 0:
                total_events[j].extend(next_gen[j])
        
        # check if there is new generated offspring
        count = np.amax([len(next_gen[j]) for j in range(d)])
        
    # sort event times    
    for j in range(d):
        total_events[j].sort()
    
    # initialize vector for lambda for M_ival values on [0,t]
    lambda_vec = np.zeros((d,M_ival))
    
    if include_lambda == True:
        
        # discretize interval [0,t]
        s_v = np.linspace(0,t,M_ival)
        
        # loop over all target components
        for i in range(d):
            
            # loop over time interval
            for v in range(M_ival):
                # base rate
                lambda_vec[i,v] = l_inf[i]
                        
                # loop over all source components
                for j in range(d):
                    
                    # loop over all events in the component 
                    for event in total_events[j]:
                        
                        # add intensity jumps and consequent decay to \lambda_i(v) at appropriate v in [0,t]
                        if event <= s_v[v]:
                            event_index = total_events[j].index(event)
                            jump_size = total_jump_sizes[i][str(event)]
                            lambda_vec[i,v] += jump_size*g_exp(i,s_v[v] - total_events[j][event_index], alpha)
                                
    return total_events, lambda_vec


def live_events(t, events, d, mu):
    
    ''' Function to determine if events are still in the system at time t. '''
    
    # check if dimensions match
    if len(events) != d:
        print("Input should be d-dimensional array")
        return None

    # check if event is still in the system at time t
    for i in range(d):
        for n in range(len(events[i])):
            s = np.random.exponential(1/mu[i])
            if events[i][n] + s < t:
                events[i][n] = 0
                
    # remove non-live events
    new_events = [[] for i in range(d)]
    for i in range(d):
        events[i] = np.asarray(events[i])
        new_events[i] = events[i][events[i] != 0]
        new_events[i] = new_events[i].tolist()
        
    return new_events


def simulate_Q(t, fixed_params, model_params):
    ''' Simulation of the Hawkes population process. Returns the d-dimensional vector
        Q(t) = (Q_1(t),...,Q_d(t)) of event arrival times that are still in the system at time t.'''
    d = model_params[0]
    mu = model_params[4]
    return live_events(t,simulate_NL(t,fixed_params, model_params)[0],d,mu)



In [2]:
''' Simulation of moments in trivariate setting '''

def averages_QL_tri(t, fixed_params, model_params, num_sims):
    ''' Returns first order moments of both Q and Lambda with size d_tri '''
    
    M = fixed_params[1]
    sims_N = [[] for n in range(num_sims)]
    sims_L = np.zeros((3, num_sims,M))
    
    for k in tqdm(range(num_sims),position=0,leave=True):
        N, L = simulate_NL(t, fixed_params, model_params)
        sims_N[k] = N
        sims_L[0][k] = L[0]
        sims_L[1][k] = L[1]
        sims_L[2][k] = L[2]
        
    sims_Q = np.zeros((3,num_sims))
    sims_L_val = np.zeros((3,num_sims))
    
    
    avgs_Q = np.zeros(3)
    avgs_L = np.zeros(3)

    for k in tqdm(range(num_sims),position=0,leave=True):
        new_N = [[] for i in range(model_params[0])]
        for i in range(model_params[0]):
            for event in range(len(sims_N[k][i])):
                if sims_N[k][i][event] < t:
                    new_N[i].append(sims_N[k][i][event])
        temp_Q = live_events(t, new_N, model_params[0], model_params[4])
        sims_Q[0][k] = len(temp_Q[0])
        sims_Q[1][k] = len(temp_Q[1])
        sims_Q[2][k] = len(temp_Q[2])

        sims_L_val[0][k] = sims_L[0][k][-1] 
        sims_L_val[1][k] = sims_L[1][k][-1] 
        sims_L_val[2][k] = sims_L[2][k][-1] 

    avgs_Q[0] = np.average(sims_Q[0],axis=0)
    avgs_Q[1] = np.average(sims_Q[1],axis=0)
    avgs_Q[2] = np.average(sims_Q[2],axis=0)

    avgs_L[0] = np.average(sims_L_val[0],axis=0)
    avgs_L[1] = np.average(sims_L_val[1],axis=0)
    avgs_L[2] = np.average(sims_L_val[2],axis=0)
    
    return avgs_Q, avgs_L



def secondmoments_QL_tri(t, fixed_params, model_params, num_sims):
    ''' Returns second order moments of both Q and Lambda in vectors of size d_tri '''
    
    M = fixed_params[1]
    sims_N = [[] for n in range(num_sims)]
    sims_L = np.zeros((3,num_sims,M))
    
    #for k in tqdm(range(num_sims),position=0,leave=True):
    for k in range(num_sims):
        N, L = simulate_NL(t, fixed_params, model_params)
        sims_N[k] = N
        sims_L[0][k] = L[0]
        sims_L[1][k] = L[1]
        sims_L[2][k] = L[2]
        
    sims_Q = np.zeros((6,num_sims))
    sims_QL_val = np.zeros((9,num_sims))
    sims_L_val = np.zeros((6,num_sims))
    
    avgs_Q = np.zeros(6)
    avgs_QL = np.zeros(9)
    avgs_L = np.zeros(6)
    
        
    #for k in tqdm(range(num_sims),position=0,leave=True):
    for k in range(num_sims):
        new_N = [[] for i in range(model_params[0])]
        for i in range(model_params[0]):
            for event in range(len(sims_N[k][i])):
                if sims_N[k][i][event] < t:
                    new_N[i].append(sims_N[k][i][event])
        temp_Q = live_events(t, new_N, model_params[0], model_params[4])

        sims_Q[0][k] = len(temp_Q[0])*(len(temp_Q[0]) - 1)
        sims_Q[1][k] = len(temp_Q[0])*len(temp_Q[1])
        sims_Q[2][k] = len(temp_Q[0])*len(temp_Q[2])
        sims_Q[3][k] = len(temp_Q[1])*(len(temp_Q[1]) - 1)
        sims_Q[4][k] = len(temp_Q[1])*len(temp_Q[2])
        sims_Q[5][k] = len(temp_Q[2])*(len(temp_Q[2]) - 1) 

        sims_QL_val[0][k] = len(temp_Q[0])*sims_L[0][k][-1]
        sims_QL_val[1][k] = len(temp_Q[1])*sims_L[0][k][-1]
        sims_QL_val[2][k] = len(temp_Q[2])*sims_L[0][k][-1]
        sims_QL_val[3][k] = len(temp_Q[0])*sims_L[1][k][-1]
        sims_QL_val[4][k] = len(temp_Q[1])*sims_L[1][k][-1]
        sims_QL_val[5][k] = len(temp_Q[2])*sims_L[1][k][-1]
        sims_QL_val[6][k] = len(temp_Q[0])*sims_L[2][k][-1]
        sims_QL_val[7][k] = len(temp_Q[1])*sims_L[2][k][-1]
        sims_QL_val[8][k] = len(temp_Q[2])*sims_L[2][k][-1]

        sims_L_val[0][k] = sims_L[0][k][-1]**2
        sims_L_val[1][k] = sims_L[0][k][-1]*sims_L[1][k][-1]
        sims_L_val[2][k] = sims_L[0][k][-1]*sims_L[2][k][-1]
        sims_L_val[3][k] = sims_L[1][k][-1]**2
        sims_L_val[4][k] = sims_L[1][k][-1]*sims_L[2][k][-1]
        sims_L_val[5][k] = sims_L[2][k][-1]**2
            
    avgs_Q[0] = np.average(sims_Q[0],axis=0)
    avgs_Q[1] = np.average(sims_Q[1],axis=0)
    avgs_Q[2] = np.average(sims_Q[2],axis=0)
    avgs_Q[3] = np.average(sims_Q[3],axis=0)
    avgs_Q[4] = np.average(sims_Q[4],axis=0)
    avgs_Q[5] = np.average(sims_Q[5],axis=0)


    avgs_QL[0] = np.average(sims_QL_val[0],axis=0)
    avgs_QL[1] = np.average(sims_QL_val[1],axis=0)
    avgs_QL[2] = np.average(sims_QL_val[2],axis=0)
    avgs_QL[3] = np.average(sims_QL_val[3],axis=0)
    avgs_QL[4] = np.average(sims_QL_val[4],axis=0)
    avgs_QL[5] = np.average(sims_QL_val[5],axis=0)
    avgs_QL[6] = np.average(sims_QL_val[6],axis=0)
    avgs_QL[7] = np.average(sims_QL_val[7],axis=0)
    avgs_QL[8] = np.average(sims_QL_val[8],axis=0)


    avgs_L[0] = np.average(sims_L_val[0],axis=0)
    avgs_L[1] = np.average(sims_L_val[1],axis=0)
    avgs_L[2] = np.average(sims_L_val[2],axis=0)
    avgs_L[3] = np.average(sims_L_val[3],axis=0)
    avgs_L[4] = np.average(sims_L_val[4],axis=0)
    avgs_L[5] = np.average(sims_L_val[5],axis=0)
    
    return avgs_Q, avgs_QL, avgs_L

In [3]:
''' Simulation of moments in bivariate setting '''

def firstmoments_QL_bi(t, fixed_params, model_params, num_sims):
    
    sims_Q = np.zeros((2,num_sims))
    sims_L = np.zeros((2,num_sims))
    
    
    for k in tqdm(range(num_sims),position=0,leave=True):
        N, L = simulate_NL(t, fixed_params, model_params)
        Q = live_events(t, N, model_params[0], model_params[4])

        sims_Q[0][k] = len(Q[0])
        sims_Q[1][k] = len(Q[1])
        
        sims_L[0][k] = L[0][-1]
        sims_L[1][k] = L[1][-1]        
        
    Q_avg = np.average(sims_Q, axis=1)
    L_avg = np.average(sims_L, axis=1)
    
    return Q_avg, L_avg


def secondmoments_QL_bi(t, fixed_params, model_params, num_sims):
    
    sims_Q = np.zeros((3,num_sims))
    sims_QL = np.zeros((4,num_sims))
    sims_L = np.zeros((3,num_sims))
    
    
    for k in tqdm(range(num_sims),position=0,leave=True):
        N, L = simulate_NL(t, fixed_params, model_params)
        Q = live_events(t, N, model_params[0], model_params[4])

        sims_Q[0][k] = len(Q[0])**2
        sims_Q[1][k] = len(Q[0])*len(Q[1])
        sims_Q[2][k] = len(Q[1])**2
        
        sims_QL[0][k] = len(Q[0])*L[0][-1]
        sims_QL[1][k] = len(Q[0])*L[1][-1]
        sims_QL[2][k] = len(Q[1])*L[0][-1]
        sims_QL[3][k] = len(Q[1])*L[1][-1]
        
        
        sims_L[0][k] = L[0][-1]**2
        sims_L[1][k] = L[0][-1]*L[1][-1]
        sims_L[2][k] = L[1][-1]**2
        
        
    Q_avg = np.average(sims_Q, axis=1)
    QL_avg = np.average(sims_QL, axis=1)
    L_avg = np.average(sims_L, axis=1)
    
    return Q_avg, QL_avg, L_avg



def thirdmoments_QL_bi(t, fixed_params, model_params, num_sims):
    
    sims_L = np.zeros((4,num_sims))
    sims_QL2 = np.zeros((6,num_sims))
    sims_Q2L = np.zeros((6,num_sims))
    sims_Q = np.zeros((4,num_sims))
    
    
    #for k in tqdm(range(num_sims),position=0,leave=True):
    for k in range(num_sims):
        N, L = simulate_NL(t, fixed_params, model_params)
        Q = live_events(t, N, model_params[0], model_params[4])
        
        # Lambda third order
        sims_L[0][k] = L[0][-1]**3
        sims_L[1][k] = L[0][-1]**2 * L[1][-1]
        sims_L[2][k] = L[0][-1] * L[1][-1]**2
        sims_L[3][k] = L[1][-1]**3
        
        # Q first order, Lambda second order
        sims_QL2[0][k] = len(Q[0])*L[0][-1]**2
        sims_QL2[1][k] = len(Q[0])*L[0][-1]* L[1][-1]
        sims_QL2[2][k] = len(Q[0])*L[1][-1]**2
        sims_QL2[3][k] = len(Q[1])*L[0][-1]**2
        sims_QL2[4][k] = len(Q[1])*L[0][-1]* L[1][-1]
        sims_QL2[5][k] = len(Q[1])*L[1][-1]**2
        
        # Q second order, Lambda first order
        sims_Q2L[0][k] = len(Q[0]) * (len(Q[0])-1) *L[0][-1]
        sims_Q2L[1][k] = len(Q[0]) * (len(Q[0])-1) *L[1][-1]
        sims_Q2L[2][k] = len(Q[0])*len(Q[1]) * L[0][-1]
        sims_Q2L[3][k] = len(Q[0])*len(Q[1]) * L[1][-1]
        sims_Q2L[4][k] = len(Q[1]) * (len(Q[1])-1) * L[0][-1]
        sims_Q2L[5][k] = len(Q[1]) * (len(Q[1])-1)* L[1][-1]
        
        # Q third order: factorial moments!
        sims_Q[0][k] = len(Q[0]) * (len(Q[0]) - 1) * (len(Q[0]) - 2)
        sims_Q[1][k] = len(Q[0]) * (len(Q[0]) - 1) * len(Q[1])
        sims_Q[2][k] = len(Q[1]) * (len(Q[1]) - 1) * len(Q[0])
        sims_Q[3][k] = len(Q[1]) * (len(Q[1]) - 1) * (len(Q[1]) - 2)
               
        
    Q_avg = np.average(sims_Q, axis=1)
    Q2L_avg = np.average(sims_Q2L, axis=1)
    QL2_avg = np.average(sims_QL2, axis=1)
    L_avg = np.average(sims_L, axis=1)
    
    return Q_avg, Q2L_avg, QL2_avg, L_avg



In [4]:
''' Simulation for auto-covariance (or auto-correlation) in bivariate setting '''


def live_events_draw_ttau(t, t_n, events, d, mu):
    # Note that t_n must be in the interval [t,t+tau] !
    
    # check if dimensions match
    if len(events) != d:
        print("Input should be d-dimensional array")
        return None
    
    events = np.array([np.array(events[i]) for i in range(d)], dtype=object)
    temp_events_t = [events[i][events[i] < t] for i in range(d)]
    temp_events_ttau = events


    # check if event is still in the system at time t
    for i in range(d):
        for n in range(len(events[i])):
            if events[i][n] < t:
                s = np.random.exponential(1/mu[i])
                if events[i][n] + s < t:
                    temp_events_t[i][n] = 0
                    temp_events_ttau[i][n] = 0
                if events[i][n] + s < t_n:
                    temp_events_ttau[i][n] = 0
            if events[i][n] > t:
                s = np.random.exponential(1/mu[i])
                if events[i][n] + s < t_n:
                    temp_events_ttau[i][n] = 0
                
    # remove non-live events
    new_events_t = [[] for i in range(d)]
    new_events_ttau = [[] for i in range(d)]
    
    for i in range(d):
        temp_events_t[i] = np.asarray(temp_events_t[i])
        new_events_t[i] = temp_events_t[i][temp_events_t[i] != 0]
        new_events_t[i] = new_events_t[i].tolist()

        temp_events_ttau[i] = np.asarray(temp_events_ttau[i])
        new_events_ttau[i] = temp_events_ttau[i][temp_events_ttau[i] != 0]
        new_events_ttau[i] = new_events_ttau[i].tolist()

    return new_events_t, new_events_ttau

def averages_QL_bi_vec_ttau(t,tau, fixed_params, model_params, num_sims, ivls):
    d = model_params[0]
    M = fixed_params[1]
    mu = model_params[4]
    
    
    #sims_N_t = [[] for n in range(num_sims)]
    sims_N_ttau = [[] for n in range(num_sims)]
    sims_L1 = np.zeros((num_sims,M))
    sims_L2 = np.zeros((num_sims,M))
    
    sims_Q1_t = np.zeros((ivls,num_sims))
    sims_Q1_ttau = np.zeros((ivls,num_sims))
    sims_Q2_t = np.zeros((ivls,num_sims))
    sims_Q2_ttau = np.zeros((ivls,num_sims))
    
    avgs_Q1Q1_prod = np.zeros(ivls)
    avgs_Q1Q2_prod = np.zeros(ivls)
    avgs_Q2Q1_prod = np.zeros(ivls)
    avgs_Q2Q2_prod = np.zeros(ivls)
    
    sims_L1_val_t = np.zeros((ivls,num_sims))
    sims_L1_val_ttau = np.zeros((ivls,num_sims))
    sims_L2_val_t = np.zeros((ivls,num_sims))
    sims_L2_val_ttau = np.zeros((ivls,num_sims))
    
    avgs_L1L1_prod = np.zeros(ivls)
    avgs_L1L2_prod = np.zeros(ivls)
    avgs_L2L1_prod = np.zeros(ivls)
    avgs_L2L2_prod = np.zeros(ivls)
    
    index_Mspace = np.linspace(0,t+tau,M)
    index_tinM = int((t/(t+tau))*M)
    timespace_t = np.linspace(t,t+tau,ivls)

    for k in tqdm(range(num_sims),position=0,leave=True):
        N, L = simulate_NL(t+tau, fixed_params, model_params)
        N = np.array([np.array(N[i]) for i in range(d)], dtype=object)
        sims_N_ttau[k] = N
        sims_L1[k] = L[0]
        sims_L2[k] = L[1]
    
    for n in tqdm(range(ivls),position=0,leave=True):
        L_index_t = index_tinM
        L_index_ttau = index_tinM
        if n > 0:
            L_index_ttau =  int(index_tinM + (n/ivls)*(M-index_tinM))

        for k in range(num_sims):
            sims_N_ttau[k] = np.array([np.array(sims_N_ttau[k][i]) for i in range(d)], dtype=object)
            new_N_ttau = [sims_N_ttau[k][i][sims_N_ttau[k][i] < timespace_t[n]] for i in range(d)]
            
            tempQ_t, tempQ_ttau = live_events_draw_ttau(t, timespace_t[n], new_N_ttau, d, mu)
            sims_Q1_t[n][k] = len(tempQ_t[0])
            sims_Q1_ttau[n][k]  = len(tempQ_ttau[0])
            sims_Q2_t[n][k] = len(tempQ_t[1])
            sims_Q2_ttau[n][k]  = len(tempQ_ttau[1])
            
            sims_L1_val_t[n][k] = sims_L1[k][L_index_t]
            sims_L1_val_ttau[n][k] = sims_L1[k][L_index_ttau]
            sims_L2_val_t[n][k] = sims_L2[k][L_index_t]
            sims_L2_val_ttau[n][k] = sims_L2[k][L_index_ttau]

        avgs_Q1Q1_prod[n] = np.average(np.multiply(sims_Q1_t[n],sims_Q1_ttau[n]),axis=0)
        avgs_Q1Q2_prod[n] = np.average(np.multiply(sims_Q1_t[n],sims_Q2_ttau[n]),axis=0)
        avgs_Q2Q1_prod[n] = np.average(np.multiply(sims_Q2_t[n],sims_Q1_ttau[n]),axis=0)
        avgs_Q2Q2_prod[n] = np.average(np.multiply(sims_Q2_t[n],sims_Q2_ttau[n]),axis=0)
        
        avgs_L1L1_prod[n] = np.average(np.multiply(sims_L1_val_t[n],sims_L1_val_ttau[n]),axis=0)
        avgs_L1L2_prod[n] = np.average(np.multiply(sims_L1_val_t[n],sims_L2_val_ttau[n]),axis=0)
        avgs_L2L1_prod[n] = np.average(np.multiply(sims_L2_val_t[n],sims_L1_val_ttau[n]),axis=0)
        avgs_L2L2_prod[n] = np.average(np.multiply(sims_L2_val_t[n],sims_L2_val_ttau[n]),axis=0)
        
    return avgs_Q1Q1_prod, avgs_Q1Q2_prod, avgs_Q2Q1_prod, avgs_Q2Q2_prod, avgs_L1L1_prod, avgs_L1L2_prod,avgs_L2L1_prod,avgs_L2L2_prod

